In [80]:
import numpy as np
import heapq

In [81]:
path = r'C:\AOC\2023\Day_17\test_data.txt'
# path = r'C:\AOC\2023\Day_17\data.txt'

In [82]:
with open(path, 'r') as file:
    input = [[int(ch) for ch in i] for i in file.read().splitlines()]

In [83]:
class Node:
    def __init__(self, coordinates, cost):
        self.coordinates = coordinates
        self.cost = cost
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)

    def __repr__(self):
        return f"Node({self.coordinates}, Cost: {self.cost})"
    
    def __str__(self):
        return f'{self.cost}'
    
    def __lt__(self, other):
        return self.cost < other.cost
    

class Grid:
    def __init__(self, ascii_map):
        self.grid = self.create_grid(ascii_map)
        self.link_nodes()

    def create_grid(self, ascii_map):
        # Assumes ASCII map is a list of strings, where each string is a row
        node_grid = []
        for x, row in enumerate(ascii_map):
            node_row = []
            for y, char in enumerate(row):
                cost = char  # Assign a default cost or based on specific characters
                node_row.append(Node((x, y), cost))
            node_grid.append(node_row)
        return node_grid

    def link_nodes(self):
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Right, Down, Left, Up
        for x in range(len(self.grid)):
            for y in range(len(self.grid[x])):
                node = self.grid[x][y]
                for dx, dy in directions:
                    new_x, new_y = x + dx, y + dy
                    if 0 <= new_x < len(self.grid) and 0 <= new_y < len(self.grid[0]):
                        child_node = self.grid[new_x][new_y]
                        node.add_child(child_node)

    def get_node(self, position):
        x, y = position
        return self.grid[x][y]

    def __getitem__(self, index):
        return self.grid[index]  

    def __repr__(self):
        return "\n".join([" ".join([repr(cell) for cell in row]) for row in self.grid])
    
    def __str__(self):
        return "\n".join([" ".join([str(cell) for cell in row]) for row in self.grid])




In [84]:
grid = Grid(input)

print(grid)

2 4 1 3 4 3 2 3 1 1 3 2 3
3 2 1 5 4 5 3 5 3 5 6 2 3
3 2 5 5 2 4 5 6 5 4 2 5 4
3 4 4 6 5 8 5 8 4 5 4 5 2
4 5 4 6 6 5 7 8 6 7 5 3 6
1 4 3 8 5 9 8 7 9 8 4 5 4
4 4 5 7 8 7 6 9 8 7 7 6 6
3 6 3 7 8 7 7 9 7 9 6 5 3
4 6 5 4 9 6 7 9 8 6 8 8 7
4 5 6 4 6 7 9 9 8 6 4 5 3
1 2 2 4 6 8 6 8 6 5 5 6 3
2 5 4 6 5 4 8 8 8 7 7 3 5
4 3 2 2 6 7 4 6 5 5 5 3 3


In [85]:
def check_four_consecutive_moves(path):
    if len(path) < 5:
        return False  # Not enough moves to have four consecutive in the same direction

    # Reverse the path and take the last 5 elements from it
    rev_path = path[::-1][:5]
    
    # Calculate directions for the last four segments of the original path
    directions = [(rev_path[i][0] - rev_path[i+1][0], rev_path[i][1] - rev_path[i+1][1]) for i in range(4)]
    print(directions)
    # Check if all four moves are in the same direction
    if all(direction == directions[0] for direction in directions):
        print(path, 'True')
        return True  # Invalid path as it contains four consecutive moves in the same direction
    
    return False
    

# use node values as g(n)
def ucs(grid, start, end):
    # visited contains nodes we have popped off priority queue
    visited = set()
    # priority queue contains tuple of states to be expanded. tuple is in the form of (g_cost, node state -> (node, path))
    fringe = []
    # initizalize empty list to store path for each particular node state
    path = [start.coordinates]
    # root_cost is the g_cost of the start state, in this case there is no cost at the start node
    root_cost = 0
    # root_state
    root_state = (start, path)
    # push root_cost(g_cost of the start state) and root_state(start state) on to priority queue
    heapq.heappush(fringe, (root_cost, root_state))
    
    while fringe:
        g_cost, node_state = heapq.heappop(fringe)
        node, path = node_state
        
        if node.coordinates == end.coordinates:
            return g_cost, path
       
        if tuple(node.coordinates) not in visited:
            visited.add(tuple(node.coordinates))
            
       # evaluate the child nodes of current node and add to stack if isn't in visited
            for child in node.children:
                if tuple(child.coordinates) not in visited:
                    new_path = path + [child.coordinates]
                    new_g_cost = g_cost + child.cost

                    if not check_four_consecutive_moves(new_path):
                        heapq.heappush(fringe, (new_g_cost, (child, new_path)))

                    # heapq.heappush(fringe, (new_g_cost, (child, new_path)))

    return None

In [86]:
def main(data, part_two=False):
    root_node = data[0][0]
    end_node = data[-1][-1]

    heatloss, path = ucs(data, root_node, end_node)
    
    return heatloss, path

In [87]:
total, path = main(grid, part_two = False)

[(0, 1), (1, 0), (0, 1), (0, 1)]
[(1, 0), (1, 0), (0, 1), (0, 1)]
[(0, 1), (1, 0), (0, 1), (1, 0)]
[(1, 0), (1, 0), (0, 1), (1, 0)]
[(0, 1), (0, 1), (0, 1), (0, 1)]
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)] True
[(1, 0), (0, 1), (0, 1), (0, 1)]
[(0, 1), (1, 0), (1, 0), (1, 0)]
[(1, 0), (1, 0), (1, 0), (1, 0)]
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)] True
[(0, 1), (1, 0), (1, 0), (0, 1)]
[(1, 0), (1, 0), (1, 0), (0, 1)]
[(0, 1), (0, 1), (1, 0), (0, 1)]
[(1, 0), (0, 1), (1, 0), (0, 1)]
[(0, 1), (1, 0), (1, 0), (0, 1)]
[(1, 0), (1, 0), (1, 0), (0, 1)]
[(0, 1), (0, 1), (0, 1), (1, 0)]
[(1, 0), (0, 1), (0, 1), (1, 0)]
[(-1, 0), (0, 1), (0, 1), (1, 0)]
[(0, 1), (1, 0), (1, 0), (1, 0)]
[(1, 0), (1, 0), (1, 0), (1, 0)]
[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2)] True
[(0, 1), (1, 0), (1, 0), (1, 0)]
[(1, 0), (1, 0), (1, 0), (1, 0)]
[(0, 0), (1, 0), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)] True
[(0, -1), (1, 0), (1, 0), (1, 0)]
[(0, 1), (1, 0), (0, 1), (1, 0)]
[(1, 0), (1, 0), (0, 1)

In [88]:
print(total)

110


In [89]:
print(path)

[(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (0, 4), (0, 5), (0, 6), (0, 7), (1, 7), (1, 8), (1, 9), (2, 9), (2, 10), (3, 10), (4, 10), (4, 11), (5, 11), (5, 12), (6, 12), (7, 12), (8, 12), (8, 11), (9, 11), (9, 12), (10, 12), (11, 12), (12, 12)]
